# Seminar on recurrent neural networks
During this seminar, we will train LSTM to solve sentiment analysis task, i. e. predict sentiment label for the text.

The recurrent neural networks work with the input of arbitrary length. However, in the implementation, it is usually much simpler to fix the sequence length (even in pytorch with its dynamic graphs :) So we will crop the sequences so that they have fixed length.

During completing this task, you will train LSTM with different level of "black box" (from using torch.nn.LSTM to implementing layer yourself). Also, you will learn different ways of applying dropout to the gated RNNs (in the RNNs there are more places where to insert binary dropout mask than in feed-forward networks).

The task can be completed on CPU but you will feel more comfortable with GPU. You may try using [https://colab.research.google.com](https://colab.research.google.com).

### Hyperparameters

In [1]:
vocab_size = 20000 
index_from = 3
n_hidden = 32 # 128
n_emb = 32 # 128
seq_len = 32 # 200
# small network on small data for seminar purposes
# after # normal size goes

batch_size = 128
learning_rate = 0.001
num_epochs = 30

### Data loading
Function load_matrix_imdb downloads data, preprocesses it and returns numpy-arrays. 

If you don't have wget, please download [archive imdb.npz](https://s3.amazonaws.com/text-datasets/imdb.npz)

In [2]:
#from rnn_utils import load_matrix_imdb
import numpy as np
import torch
import torch.utils.data
import os
import re
from collections import defaultdict
import operator

In [8]:
def load_matrix_imdb(path='imdb.npz', num_words=None, skip_top=0,
              maxlen=None, seed=113,
              start_char=1, oov_char=2, index_from=3, **kwargs):
    """
    Modified code from Keras
    Loads data matrixes from npz file, crops and pads seqs and returns
    shuffled (x_train, y_train), (x_test, y_test)
    """
    if not os.path.exists(path):
        print("Downloading matrix data into current folder")
        os.system("wget https://s3.amazonaws.com/text-datasets/imdb.npz")
        
    with np.load(path, allow_pickle=True) as f:
        x_train, labels_train = f['x_train'], f['y_train']
        x_test, labels_test = f['x_test'], f['y_test']

    np.random.seed(seed)
    indices = np.arange(len(x_train))
    np.random.shuffle(indices)
    x_train = x_train[indices]
    labels_train = labels_train[indices]

    indices = np.arange(len(x_test))
    np.random.shuffle(indices)
    x_test = x_test[indices]
    labels_test = labels_test[indices]

    xs = np.concatenate([x_train, x_test])
    labels = np.concatenate([labels_train, labels_test])

    if start_char is not None:
        xs = [[start_char] + [w + index_from for w in x] for x in xs]
    elif index_from:
        xs = [[w + index_from for w in x] for x in xs]

    if not num_words:
        num_words = max([max(x) for x in xs])
    if not maxlen:
        maxlen = max([len(x) for x in xs])

    # by convention, use 2 as OOV word
    # reserve 'index_from' (=3 by default) characters:
    # 0 (padding), 1 (start), 2 (OOV)
    xs_new = []
    for x in xs:
        x = x[:maxlen] # crop long sequences
        if oov_char is not None: # replace rare or frequent symbols 
            x = [w if (skip_top <= w < num_words) else oov_char for w in x]
        else: # or filter rare and frequent symbols
            x = [w for w in x if skip_top <= w < num_words]
        x_padded = np.zeros(maxlen)#, dtype = 'int32')
        x_padded[-len(x):] = x
        xs_new.append(x_padded)    
            
    idx = len(x_train)
    x_train, y_train = np.array(xs_new[:idx]), np.array(labels[:idx])
    x_test, y_test = np.array(xs_new[idx:]), np.array(labels[idx:])

    return (x_train, y_train), (x_test, y_test)

In [9]:
np.random.seed(0)
(X_train, y_train), (X_test, y_test) = load_matrix_imdb(num_words=vocab_size,
                                                        maxlen=seq_len)

In [10]:
set(y_train) # binary classification

{0, 1}

In [11]:
X_train.shape, X_test.shape

((25000, 32), (25000, 32))

In [12]:
X_train[0] # sequence of coded words

array([  1.00000000e+00,   1.40000000e+01,   2.20000000e+01,
         1.60000000e+01,   4.30000000e+01,   5.30000000e+02,
         9.73000000e+02,   1.62200000e+03,   1.38500000e+03,
         6.50000000e+01,   4.58000000e+02,   4.46800000e+03,
         6.60000000e+01,   3.94100000e+03,   4.00000000e+00,
         1.73000000e+02,   3.60000000e+01,   2.56000000e+02,
         5.00000000e+00,   2.50000000e+01,   1.00000000e+02,
         4.30000000e+01,   8.38000000e+02,   1.12000000e+02,
         5.00000000e+01,   6.70000000e+02,   2.00000000e+00,
         9.00000000e+00,   3.50000000e+01,   4.80000000e+02,
         2.84000000e+02,   5.00000000e+00])

In [13]:
y_train

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [14]:
train_dset = torch.utils.data.TensorDataset(torch.tensor(X_train, dtype=torch.long), 
                               torch.tensor(y_train, dtype=torch.long))

In [15]:
test_dset = torch.utils.data.TensorDataset(torch.tensor(X_test, dtype=torch.long), 
                               torch.tensor(y_test, dtype=torch.long))

In [16]:
train_loader = torch.utils.data.DataLoader(train_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

In [17]:
test_loader = torch.utils.data.DataLoader(test_dset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4
                         )

### Definind and training RNN in pytroch

In [18]:
import os
import torch.optim as optim
import torch.nn as nn

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
device

device(type='cuda')

Our RNN will process the input sequence by words (word level). We will use simple architecture consisting of embedding layer, 1 LSTM layer and fully-connected layer on the last hidden state.

The code below defines and trains the network. __Pay attention__ to "### pay attention here" marks: they point to RNN specifics.

Run this code so that you can compare training time with different models and implementations later.

In [21]:
class RNNClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, \
                 batch_size, rec_layer=nn.LSTM, embedding=nn.Embedding, \
                 dropout=None):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = embedding(vocab_size, embedding_dim)
        if dropout:
            self.rnn = rec_layer(embedding_dim, hidden_dim, dropout=dropout)
        else:
            self.rnn = rec_layer(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
    
    def forward(self, sentences):
        embedding = self.word_embeddings(sentences)
        out, hidden = self.rnn(embedding) # pay attention here!
        res = self.hidden2label(out[-1])
        return torch.sigmoid(res)
    

[LSTM dource code](http://pytorch.org/docs/master/_modules/torch/nn/modules/rnn.html#LSTM)

In [22]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=nn.LSTM,
                       dropout=None).to(device)

In [23]:
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
lossfun = nn.BCELoss(reduction='sum')

In [24]:
def train_epoch(train_loader, model, lossfun, optimizer, device):
    model.train()
    for it, traindata in enumerate(train_loader):
        train_inputs, train_labels = traindata
        train_inputs = train_inputs.to(device) 
        train_labels = train_labels.to(device)
        train_labels = torch.squeeze(train_labels)

        model.zero_grad()        
        output = model(train_inputs.t()) # pay attention here!

        loss = lossfun(output.view(-1), train_labels.float())
        loss.backward()
        optimizer.step()

def evaluate(loader, model, lossfun, device):
    model.eval()
    total_acc = 0.0
    total_loss = 0.0
    total = 0.0
    for it, data in enumerate(loader):
        inputs, labels = data
        inputs = inputs.to(device) 
        labels = labels.to(device)
        labels = torch.squeeze(labels)

        output = model(inputs.t()) # pay attention here!
        loss = lossfun(output.view(-1), labels.float())
        total_loss += loss.item()

        # calc testing acc        
        pred = output.view(-1) > 0.5
        correct = pred == labels.bool()
        total_acc += torch.sum(correct).item() / len(correct)

    total = it + 1
    return total_loss / total, total_acc / total
    

def train(train_loader, test_loader, model, lossfun, optimizer, \
          device, num_epochs):
    train_loss_ = []
    test_loss_ = []
    train_acc_ = []
    test_acc_ = []
    for epoch in range(num_epochs):
        train_epoch(train_loader, model, lossfun, optimizer, device)
        train_loss, train_acc = evaluate(train_loader, model, lossfun, device)
        train_loss_.append(train_loss)
        train_acc_.append(train_acc)
        test_loss, test_acc = evaluate(test_loader, model, lossfun, device)
        test_loss_.append(test_loss)
        test_acc_.append(test_acc)

        print(f'Epoch: {epoch+1:3d}/{num_epochs:3d} '
              f'Training Loss: {train_loss_[epoch]:.3f}, Testing Loss: {test_loss_[epoch]:.3f}, '
              f'Training Acc: {train_acc_[epoch]:.3f}, Testing Acc: {test_acc_[epoch]:.3f}')

    return train_loss_, train_acc_, test_loss_, test_acc_

In [26]:
%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 43.802, Testing Loss: 79.342, Training Acc: 0.851, Testing Acc: 0.714
Epoch:   2/ 30 Training Loss: 40.148, Testing Loss: 78.565, Training Acc: 0.874, Testing Acc: 0.711
Epoch:   3/ 30 Training Loss: 36.081, Testing Loss: 84.720, Training Acc: 0.885, Testing Acc: 0.712
Epoch:   4/ 30 Training Loss: 34.386, Testing Loss: 89.910, Training Acc: 0.892, Testing Acc: 0.707
Epoch:   5/ 30 Training Loss: 32.446, Testing Loss: 93.470, Training Acc: 0.899, Testing Acc: 0.700
Epoch:   6/ 30 Training Loss: 28.454, Testing Loss: 100.271, Training Acc: 0.914, Testing Acc: 0.709
Epoch:   7/ 30 Training Loss: 26.007, Testing Loss: 98.548, Training Acc: 0.925, Testing Acc: 0.707
Epoch:   8/ 30 Training Loss: 24.852, Testing Loss: 108.762, Training Acc: 0.925, Testing Acc: 0.701
Epoch:   9/ 30 Training Loss: 21.043, Testing Loss: 108.376, Training Acc: 0.942, Testing Acc: 0.707
Epoch:  10/ 30 Training Loss: 19.584, Testing Loss: 110.469, Training Acc: 0.947, Testing Acc: 0.

Unregularized LSTM often overfits (and we see that test accuracy degrates). To overcome it, L2 regularization and droput are usually applied. But there are several ways how to apply dropout to gated RNNs, and not all of them work well. Please refer to this [blog post](https://medium.com/@bingobee01/a-review-of-dropout-as-applied-to-rnns-72e79ecd5b7b) for good review.

We will implement three dropouts for LSTM. While doing so, we will see that for different methods we need to "reveal" the layer to the different "depth" (from black-box to implementing layer ourselves).

### Dropout by (Gal and Ghahramani)

Let's start with a dropout proposed by [Gal and Ghahramani](https://arxiv.org/abs/1512.05287).

To implement it, we have to use nn.LSTMCell (processes 1 time step) instead of nn.LSTM (processes the whole input sequence). 

Complete class RNNLayer. With dropout=0 it has to work as usual LSTM, and with dropout > 0 it has to multiply the input and hidden vector by random binary mask, and this mask should be __the same for all time steps__.

Formulas for this dropout (m denotes applying dropout):
$$
h_{t-1} = h_{t-1} \odot m_h, \, x_t = x_t \odot m_x
$$
(after this, usual LSTM step goes)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$

In [27]:
def init_h0_c0(num_objects, hidden_size, some_existing_tensor):
    """
    return h0 and c0, use some_existing_tensor.new_zeros() to gen them
    h0 shape: num_objects x hidden_size
    c0 shape: num_objects x hidden_size
    """
    h0 = some_existing_tensor.new_zeros((num_objects , hidden_size))
    c0 = some_existing_tensor.new_zeros((num_objects , hidden_size))
    return h0 , c0
    

In [42]:
def gen_dropout_mask(input_size, hidden_size, is_training, p, some_existing_tensor):
    """
    is_training: if True, gen masks from Bernoulli
                 if False, gen masks consisting of (1-p)
    
    return two dropout masks of sizes (input_size, ), (hidden_size, )
    if p is not None
    return one masks if p is None
    """
    if(is_training):
        input_mask = some_existing_tensor.new_zeros((input_size)).bernoulli(p)
        hidden_mask = some_existing_tensor.new_zeros((hidden_size)).bernoulli(p)
    else:
        input_mask = some_existing_tensor.new_zeros((input_size)).fill_(1 - p)
        hidden_mask = some_existing_tensor.new_zeros((hidden_size)).fill_(1 - p)
    if (p):
        return input_mask, input_mask
    else:
        input_mask = some_existing_tensor.new_zeros((input_size)).bernoulli(1)
        hidden_mask = some_existing_tensor.new_zeros((hidden_size)).bernoulli(1)
        return input_mask, input_mask
        
    #...new_zeros(...).bernoulli(...)

In [43]:
class RNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=None):
        super(RNNLayer, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.dropout = dropout
        self.rnn_cell = nn.LSTMCell(input_size, hidden_size)
        
    def forward(self, inp):
        # initialize h_0, c_0
        h_i, c_i = init_h0_c0(inp.shape[1], self.hidden_size, inp)
        
        # gen masks
        input_mask, hidden_mask = gen_dropout_mask(self.input_size, \
                                                   self.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   inp)
        
        out = []
        out.append(h_i)
        for i in range (inp.shape[0]):
            h_i , c_i = self.rnn_cell(inp[i] * input_mask , (h_i * hidden_mask , c_i) )
            out.append(h_i)
            h_i *= hidden_mask
        return out , (h_i,c_i);
        ### your code here
        ### implement recurrent logic and return what nn.LSTM returns
        ### do not forget to apply generated dropout masks!

class RNNClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, \
                 batch_size, rec_layer=RNNLayer, embedding=nn.Embedding, \
                 dropout=None):
        super(RNNClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = embedding(vocab_size, embedding_dim)
        if dropout:
            self.rnn = rec_layer(embedding_dim, hidden_dim, dropout=dropout)
        else:
            self.rnn = rec_layer(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
    
    def forward(self, sentences):
        embedding = self.word_embeddings(sentences)
        out, hidden = self.rnn(embedding) # pay attention here!
        res = self.hidden2label(out[-1])
        return torch.sigmoid(res)

In [44]:
model = RNNClassifier(embedding_dim=n_emb,
                       hidden_dim=n_hidden,
                       vocab_size=vocab_size,
                       label_size=1,
                       batch_size=batch_size, 
                       rec_layer=RNNLayer,
                       dropout=0.5).to(device)


Test your implementation with dropout turned off (pass RNNLayer to RNNClassifier as rec_layer). Measure the training time (%time). Does it differ from training time of nn.LSTM?

In [45]:
%time a, b, c, d = train(train_loader, test_loader, model, lossfun, \
                   optimizer, device, num_epochs)

Epoch:   1/ 30 Training Loss: 88.427, Testing Loss: 88.432, Training Acc: 0.504, Testing Acc: 0.502
Epoch:   2/ 30 Training Loss: 88.427, Testing Loss: 88.432, Training Acc: 0.504, Testing Acc: 0.502
Epoch:   3/ 30 Training Loss: 88.427, Testing Loss: 88.432, Training Acc: 0.504, Testing Acc: 0.502
Epoch:   4/ 30 Training Loss: 88.427, Testing Loss: 88.432, Training Acc: 0.504, Testing Acc: 0.501
Epoch:   5/ 30 Training Loss: 88.427, Testing Loss: 88.432, Training Acc: 0.504, Testing Acc: 0.502
Epoch:   6/ 30 Training Loss: 88.427, Testing Loss: 88.432, Training Acc: 0.504, Testing Acc: 0.502
Epoch:   7/ 30 Training Loss: 88.427, Testing Loss: 88.432, Training Acc: 0.504, Testing Acc: 0.501


KeyboardInterrupt: 

Test your implementation with dropout=0.5 again measuring the training time. Does the model still overfit? Does the training take more time than training without dropout? (additional time is spent for mask generating)

### Dropout by (Gal and Ghahramani). Second try

< start hacking pytorch >

When you unroll the time  cycle in python, training slows down. But there is the way how to implement dropout by (Gal and Ghahramani) without modifying computational graph and modifying only weights of the network. This allows using nn.LSTM instead of nn.LSTMCell. Before calling nn.LSTM, you should replace its weights with the weights where some rows are multiplied by 0. Of course, in this case you have to store the trainable weights separately. This is the way how this dropout is implementd in FastAI library, which code  is used in the cell below.

Complete the class:

In [ ]:
import warnings

In [ ]:
def LSTMcell(in_put, (hx , cx) ):
    return 

In [ ]:
class FastRNNLayer(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0):
        super(FastRNNLayer, self).__init__()
        self.module = nn.LSTM(input_size, hidden_size)
        self.dropout = dropout
        self.layer_names = ['weight_hh_l0', 'weight_ih_l0']
        for layer in self.layer_names:
            # Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            
    def _setweights(self):
        #"Apply dropout to the raw weight
        # gen masks
        input_mask, hidden_mask = gen_dropout_mask(self.input_size, \
                                                   self.hidden_size, \
                                                   self.training, \
                                                   self.dropout, \
                                                   inp)
        
        
        for layer, mask in zip(self.layer_names, (hidden_mask, input_mask)):
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = raw_w * mask

    def forward(self, *args):
        with warnings.catch_warnings():
            # To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            
            ### your code here
            ### set new weights of self.module and call its forward
        out = []
        out.append(h_i)
        for i in range (inp.shape[0]):
            h_i , c_i = self.rnn_cell(inp[i] * input_mask , (h_i * hidden_mask , c_i) )
            out.append(h_i)
            h_i *= hidden_mask
        return out , (h_i,c_i);

    def reset(self):
        if hasattr(self.module, 'reset'): self.module.reset()

Test your implementation (again passing FastRNNLayer as a rec_layer) with dropout = 0.5. Compare training time with previous models. The test accuracy and other training metrics should be the same as with previous implementaton.

< end hacking pytorch >

### Dropout by (Semeniuta et al)
Now let's turn to implementing dropout proposed by [Semeniuta et al](http://www.aclweb.org/anthology/C16-1165). 

This method is even more popular than the previous one. It is developed specifically for _gated_ recurrent architectures. For LSTM, this dropout applies dropout only to information flow ($m_h$ is a dropout mask):
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot g \odot {\bf m_h} \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
For $x_t$, the mask is put in the same way as in (Gal and Ghahramani). By the way, you can apply this mask before passing tensor to LSTM layer.

According to the paper, the mask can be the same for all moments of the time but may also be  different. We will use the same mask.

To implement this dropout, you have to implement LSTM by yourself (interface of LSTMCell is not enough as you should work with LSTM logics). Complete the class:

In [ ]:
class HandmadeLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.):
        super(HandmadeLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.input_weights = nn.Linear(input_size, 4 * hidden_size)
        self.hidden_weights = nn.Linear(hidden_size, 4 * hidden_size)
        
        self.reset_params()


    def reset_params(self):
        """
        initialization as in Pytorch
        do not forget to call this method!
        """
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(0, stdv)
            

    def forward(self, inp, hidden=None):
        def lstmcell (inp , hidden):
            hx , cx = hidden
            inp = inp * input_mask  
            i , f , g , o = (self.input_weights(inp) ).chunck(4 , dim = 1)
    
    
            return h_x , c_x
        ### your code here
        # use functions init_h0_c0 and gen_dropout_masks defined above
        out = []
        out.append(h_i)
        for i in range (inp.shape[0]):
            h_i , c_i = lstmcell(inp[i] * input_mask , (h_i * hidden_mask , c_i) )
            out.append(h_i)
            h_i *= hidden_mask
        return out , (h_i,c_i);

Test your implementation without dropout (controlthe quality and compare the training time with nn.LSTM) and with dropout=0.5. Copare the quality with the model trained with the previous dropout.

### Zoneout
In Zoneout, at each time step you update the hidden state with probability p and hold it the same with probability 1-p. Formulas for Zoneout:
 
(firstly usual time step goes, e. g. LSTM)
$$
i = \sigma(h_{t-1}W^i + x_t U^i+b_i) \quad
o = \sigma(h_{t-1}W^o + x_t U^o+b_o) 
$$
$$
f = \sigma(h_{t-1}W^f + x_t U^f+b_f) \quad 
g = tanh(h_{t-1} W^g + x_t U^g+b_g) 
$$
$$
c_t = f \odot c_{t-1} +  i \odot  g \quad
h_t =  o \odot tanh(c_t) \nonumber
$$
Then apply Zoneout:
$$
h_t = h_t \odot m_h^t + h_{t-1}\odot(1-m_h^t)
$$
In this method, the mask should be different at different moments of the time (otherwise the method simplifies to the dropout by (Gal and Ghahramani)). For x_t you can apply dropout before using LSTM layer.

If you have time left, you may implement this method. Choose one of three our implementations as a base.